In [1]:
"""
Sasank Desaraju
12/25/2023

This notebook is to play with different code modules and see how they work
This is a repo about medical AI using Pytorch Lightning
"""

'\nSasank Desaraju\n12/25/2023\n\nThis notebook is to play with different code modules and see how they work\nThis is a repo about medical AI using Pytorch Lightning\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

/home/sasank/miniconda3/envs/fistula/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/sasank/miniconda3/envs/fistula/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# load and explore .npz file of image data
data = np.load('../data/chestmnist.npz')

In [4]:
# explore the np file of data. What fields does is contain?
print(data.files)

['train_images', 'val_images', 'test_images', 'train_labels', 'val_labels', 'test_labels']


In [6]:
train_images = data['train_images']
train_labels = data['train_labels']

In [ ]:
# Create a Lightning DataModule for the ChestMNIST dataset
class ChestMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = '../data', batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def prepare_data(self):
        # download, split, etc...
        # only called on 1 GPU/TPU in distributed
        # The data is already downloaded and is at 'data/chestmnist.npz'
    
    def setup(self, stage=None):
        # make assignments here (val/train/test split)
        # called on every process in DDP
        # stage defines if we are at fit or test step
        if stage == 'fit' or stage is None:
            data = np.load(os.path.join(self.data_dir, 'chestmnist.npz'))
            self.train_images = torch.from_numpy(data['train_images']).float()
            self.train_labels = torch.from_numpy(data['train_labels']).long()
            self.val_images = torch.from_numpy(data['val_images']).float()
            self.val_labels = torch.from_numpy(data['val_labels']).long()
        if stage == 'test' or stage is None:
            data = np.load(os.path.join(self.data_dir, 'chestmnist.npz'))
            self.test_images = torch.from_numpy(data['test_images']).float()
            self.test_labels = torch.from_numpy(data['test_labels']).long()
        
    def train_dataloader(self):
        # return a training dataloader
        return DataLoader(TensorDataset(self.train_images, self.train_labels), batch_size=self.batch_size)
    
    def val_dataloader(self):
        # return a validation dataloader
        return DataLoader(TensorDataset(self.val_images, self.val_labels), batch_size=self.batch_size)